In [1]:
import os
import numpy as np 
from joblib import Parallel, delayed

In [2]:
def check_inside_mandlebrot_bernoullii(i, c):
    z = 0 + 0j
    for i in range(i):
        z = np.square(z) + c
        if np.abs(z) > 2:
            return 0
    return 1


def check_inside_mandlebrot(i, c):
    z = 0 + 0j
    for i in range(i):
        z = np.square(z) + c
        if np.abs(z) > 2:
            return False
    return True


def mandlebrot_area(i, s, sampling_function):
    random_complex_numbers = sampling_function(s)

    in_mandlebrot = 0
 
    for z in random_complex_numbers:
        if check_inside_mandlebrot(i, z):
            in_mandlebrot += 1

    area = in_mandlebrot / s * area_box

    return area


def uniform_samples(n):
    real = np.random.uniform(r_min, r_max, size=n)
    imaginary = np.random.uniform(i_min, i_max, size=n) * 1j
    random_complex_numbers = real + imaginary
    return random_complex_numbers


In [3]:
r_min = -2
r_max = 0.46
i_min = -1.12
i_max = 1.12
area_box = (r_max - r_min) * (i_max - i_min)
default_i = 50

In [15]:
# maximun standard deviation 
max_std = 0.1

# maximum standard deviation of the probability that a point is in the mandlebrot set
d = max_std / area_box

# relates to 99.9 % that the standard deviation is indeed below that maximun
z_value = 3.27

# keep track of how many samples have been drawn
k = 0

# draw 100 random variables
points = uniform_samples(100)
values = np.zeros(100)
for i, p in enumerate(points):
    xi = check_inside_mandlebrot_bernoullii(default_i, p)
    values[i] = xi
k += 100

# compute the sample mean and sample standard deviation
sample_mean = values.mean()
sample_var = np.square(values - sample_mean).sum() / (len(values) - 1)
sample_std = np.sqrt(sample_var)

# draw until the sample standard deviation is down to an acceptable level
while z_value * sample_std / np.sqrt(k) > d:
    # add an extra random variable
    point = uniform_samples(1)
    xi = check_inside_mandlebrot_bernoullii(default_i, point)

    # recalculate the mean and variance
    new_sample_mean = sample_mean + (xi - sample_mean) / (k + 1)
    sample_var = (1 - 1 / k) * sample_var + (k + 1) * (new_sample_mean - sample_mean) ** 2
    sample_mean = new_sample_mean
    sample_std = np.sqrt(sample_var)
    k += 1

print(f'estimated area is {sample_mean * area_box}')
print(f'the std on this estimation is lower than {max_std} with 99.9% confidence')
print(f'{k} samples needed')

estimated area is 1.5522371632932532
the std on this estimation is lower than 0.1 with 99.9% confidence
6571 samples needed


In [21]:
# interval  length
l = 0.1

# specify the confidence interval as (1 - alpha) * 100 = ci
apha = 0.01

# relates to 99.9% ci as Z_alpha/2 = .327
z_value = 3.27

# keep track of how many samples have been drawn
k = 0

# draw 100 random variables
points = uniform_samples(100)
values = np.zeros(100)
for i, p in enumerate(points):
    xi = check_inside_mandlebrot_bernoullii(default_i, p)
    values[i] = xi
k += 100

# compute the sample mean and sample standard deviation
sample_mean = values.mean()
sample_var = np.square(values - sample_mean).sum() / (len(values) - 1)
sample_std = np.sqrt(sample_var)

# draw until the sample standard deviation is down to an acceptable level
while 2 * z_value * sample_std / np.sqrt(k) > l:
    # add an extra random variable
    point = uniform_samples(1)
    xi = check_inside_mandlebrot_bernoullii(default_i, point)

    # recalculate the mean and variance
    new_sample_mean = sample_mean + (xi - sample_mean) / (k + 1)
    sample_var = (1 - 1 / k) * sample_var + (k + 1) * (new_sample_mean - sample_mean) ** 2
    sample_mean = new_sample_mean
    sample_std = np.sqrt(sample_var)
    k += 1

area = sample_mean * area_box

print(f'estimated area is {area}')
print(f'we are 95% confident that the real area theta lies between {area - l} and {area + l}')
print(f'{k} samples needed')

estimated area is 1.59495119182747
we are 95% confident that the real area theta lies between 1.4949511918274698 and 1.69495119182747
881 samples needed


In [24]:
n_iterations = 10
n_cores = os.cpu_count()

n_samples = k
areas = Parallel(n_jobs=n_cores)(
    delayed(mandlebrot_area)(default_i, n_samples, uniform_samples) for _ in range(n_iterations)
)
areas = np.array(areas)

mean = areas.mean(axis=0)
std = areas.std(axis=0)

print(f'mean {mean}')
print(f'std {std}')

mean 1.6356068104426793
std 0.06772728132451791


In [7]:
# how many samples - depends on the desired accuracy
# how fast does it converse to that result, std wise
# how many iterations - depends on the ?
# how fast does it converge to that result, mean output wise
